![GenAI Banner](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)


<p><font size="5" color='grey'> <b>
Agenten (LangGraph)
</b></font> </br></p>


---

In [ ]:
#@title 🔧 Umgebung einrichten{ display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/GenAI.git#subdirectory=04_modul
from genai_lib.utilities import check_environment, get_ipinfo, setup_api_keys, mprint, install_packages
setup_api_keys(['OPENAI_API_KEY', 'HF_TOKEN', 'SERPAPI_API_KEY'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()
# Bei Bedarf: Trennen zwischen Installationsname () und Importname (für Python) beide Angaben in Klammern
    # install_packages([('markitdown[all]', 'markitdown'), 'langchain_chroma', ]

In [ ]:
#@title 🛠️ Installationen { display-mode: "form" }
install_packages([
    ('google-search-results', 'serpapi'),
    'wikipedia',
])

# 1 | Was ist ein echter Agent?
---

Wenn man sich mit generativer KI beschäftigt, stößt man früher oder später auf den Begriff **Agent** – also ein System, das Aufgaben eigenständig ausführt. Doch was genau ist ein *„echter“* Agent? Muss er vollständig autonom sein? Solche Fragen führen schnell zu endlosen Grundsatzdiskussionen – und genau das ist nicht hilfreich, wenn man einfach anfangen möchte, mit GenAI zu arbeiten.

<img src="https://raw.githubusercontent.com/ralf-42/Image/main/agentisch_1.png" class="logo" width="500"/>



Der bekannte KI-Experte **Andrew Ng** hat einen anderen, praktischeren Vorschlag gemacht: Statt darüber zu streiten, ob etwas ein Agent ist oder nicht, sollten wir lieber davon sprechen, wie **agentisch** ein System ist – also wie **selbstständig** es arbeitet. So kann man sich auf das konzentrieren, was wirklich zählt: Was kann das System leisten, und wo kann es sinnvoll eingesetzt werden?




<img src="https://raw.githubusercontent.com/ralf-42/Image/main/agentisch_2.png" class="logo" width="900"/>



Besonders einfach ist der Einstieg bei eher einfachen Aufgaben – also bei Prozessen, die heute noch manuell erledigt werden, wie das Ausfüllen von Formularen, das Nachschlagen in einer Datenbank oder das Kopieren von Informationen zwischen verschiedenen Systemen. Diese Aufgaben lassen sich gut in sogenannte agentische Workflows überführen – also in Abläufe, bei denen die KI (teilweise) selbstständig handelt.


Natürlich gibt es auch deutlich komplexere Anwendungen, bei denen die KI viele Entscheidungen trifft, Schleifen durchläuft und sich an neue Situationen anpasst. Solche Systeme sind spannend – aber gerade für den Anfang ist es oft sinnvoller, mit kleineren, überschaubaren Schritten zu starten. Dort liegen aktuell auch die meisten Chancen, GenAI im Alltag oder im Beruf sinnvoll einzusetzen.



**Fazit:**     
Es muss nicht gleich ein „superintelligenter“ Agent sein. Besser ist es, pragmatisch zu denken, einfache Prozesse zu automatisieren – und so Schritt für Schritt Erfahrungen zu sammeln.



<br>

**Agenten erweitern LLMs**

Agenten sind mehr als nur Sprachmodelle. Sie verbinden die Sprachfähigkeiten eines LLMs mit praktischer Handlungsfähigkeit:

+ LLM als Denkmodul: Das LLM übernimmt das Sprachverständnis und das logische Schlussfolgern.

+ Erweiterbarkeit durch Tools: Agenten greifen z. B. auf Web, Datenbanken oder externe APIs zu.

+ Transparenter Denkprozess: Der Agent zeigt, wie er zum Ergebnis kommt – ideal zum Lernen und Verstehen.

+ Entscheidungen treffen: Agenten analysieren, planen und wählen aus mehreren Optionen – nicht nur einmal, sondern iterativ.

# 2 | Direkter Vergleich
---

## 2.1 Setup und Tools

Bevor wir vergleichen können, müssen wir die notwendigen Tools für unseren Agenten definieren. Diese Tools repräsentieren die erweiterten Fähigkeiten, die einem einfachen LLM fehlen.



In [ ]:
# Einfache Tools definieren
from langchain_core.tools import Tool
from langchain_community.utilities.serpapi import SerpAPIWrapper
import requests

# ---- Tool 1 - calculator
def simple_calculator(expression):
    """Einfacher Rechner - Das kann ein LLM oft nicht präzise"""
    try:
        # Sicherheitscheck
        if any(x in expression for x in ['import', 'exec', '__']):
            return "Unsichere Operation"
        result = eval(expression)
        return f"{expression} = {result}"
    except:
        return "Berechnungsfehler"

# ---- Tool 2 - internet_search
serpapi = SerpAPIWrapper()

# ---- Tool 3 - weather_open_meteo
def get_weather_open_meteo(address):
    """Aktuelle Wetterdaten über Open-Meteo API"""
    try:
        geo_url = "https://nominatim.openstreetmap.org/search"
        params = {"q": address, "format": "json", "limit": 1}
        headers = {"User-Agent": "Wetterdaten-Skript/1.0 (kontakt@example.com)"} # Good practice
        geo_response = requests.get(geo_url, params=params, headers=headers).json()

        if geo_response:
            lat, lon = geo_response[0]["lat"], geo_response[0]["lon"]
            # Request Celsius temperature
            weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true&temperature_unit=celsius&windspeed_unit=kmh"
            weather_response = requests.get(weather_url).json()

            if 'current_weather' in weather_response:
                current_weather = weather_response['current_weather']
                temp_c = current_weather.get('temperature', 'N/A')
                windspeed_kmh = current_weather.get('windspeed', 'N/A')
                weathercode = current_weather.get('weathercode', 'N/A') # You might want to map weathercode to description
                return f"Wetter in {address}: Temperatur {temp_c}°C, Windgeschwindigkeit {windspeed_kmh} km/h, Wettercode: {weathercode}"
            else:
                 return f"Keine Wetterdaten gefunden für {address}"
        else:
            return f"Fehler beim Abrufen der Geodaten für {address}"
    except Exception as e:
        return f"Fehler beim Abrufen der Wetterdaten: {e}"


# Tool-Liste erstellen
tools = [
    Tool(
        name="calculator",
        func=simple_calculator,
        description="🔢 RECHNER - Präzise Berechnungen (was LLM oft falsch macht)"
    ),
    Tool(
        name="internet_search",
        func=serpapi.run,
        description="🌐 INTERNETSUCHE - Aktuelle Informationen finden (was LLM NICHT kann)"
    ),
     Tool(
        name="weather_open_meteo",
        func=get_weather_open_meteo,
        description="🌤️ WETTER (Open-Meteo) - Echtzeitdaten abrufen (was LLM unmöglich ist). Input ist die Adresse oder der Ort."
    )
]

print("✅ Tools definiert:")
for tool in tools:
    print(f"   • {tool.name:25s}: {tool.description}")

## 2.2 Vergleichstest

Wir verwenden eine Frage, die sowohl aktuelle Daten als auch eine Berechnung erfordert, um die Grenzen eines LLMs und die Stärken eines Agenten zu demonstrieren.

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
import time

# LLM Setup
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0)

# Test-Frage die Grenzen aufzeigt
test_question = "Wie ist das Wetter in Berlin, was ist 2847 * 1923, wie ist der aktuelle XETRA Kurs der Aktie von Rheinmetall?"

mprint("### 🧪 VERGLEICHSTEST")
mprint("---")
mprint(f"**Frage:** {test_question}")
print()

In [ ]:
# 1. EINFACHES LLM PROBIEREN
mprint("### 1️⃣ EINFACHES LLM:")
mprint("---")

start_time = time.time()
llm_response = llm.invoke(test_question)
llm_time = time.time() - start_time

mprint(f"**Antwort:** {llm_response.content}")
mprint(f"**Zeit:** {llm_time:.2f}s")

In [ ]:
# 2. AGENT MIT TOOLS (LangGraph)
mprint("### 2️⃣ AGENT MIT TOOLS (LangGraph):")
mprint("---")

# System-Prompt für den Agenten
system_prompt = "Du bist ein hilfreicher Agent. Nutze Tools für aktuelle Daten und Berechnungen. Formatiere das Ergebnis im Markdown-Format, Formeln: $ Formel $"

# LangGraph Agent erstellen
agent_executor = create_react_agent(llm, tools, prompt=system_prompt)

start_time = time.time()

In [ ]:
# Streaming: Zeigt den "Denkprozess" in Echtzeit
mprint("**🔄 Agent-Denkprozess (Streaming):**")
mprint("---")

final_state = None
seen_message_count = 0

for event in agent_executor.stream({"messages": [("human", test_question)]}, stream_mode="values"):
    final_state = event
    new_messages = event["messages"][seen_message_count:]
    seen_message_count = len(event["messages"])

    for msg in new_messages:
        # Tool-Aufrufe
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            for tool_call in msg.tool_calls:
                mprint(f"🔧 **Tool-Aufruf:** `{tool_call['name']}`")

        # Tool-Ergebnisse
        if msg.type == "tool":
            mprint(f"📊 **Tool-Ergebnis ({msg.name}):** {msg.content}")

agent_time = time.time() - start_time

In [ ]:
# Finale Antwort extrahieren
output_message = None
if final_state:
    for msg in reversed(final_state["messages"]):
        if msg.type == "ai" and not (hasattr(msg, 'tool_calls') and msg.tool_calls):
            output_message = msg.content
            break

print()
mprint(f"## 🤖 KI-Agent (LangGraph): ")
mprint("---")
mprint(output_message if output_message else "⚠️ Keine finale Antwort gefunden")
mprint(f"**Zeit:** {agent_time:.2f}s")

## 2.3 Unterschiede



Die Unterschiede zwischen LLM und Agent lassen sich in fünf Kernbereichen zusammenfassen, die den Paradigmenwechsel von statischer zu dynamischer KI verdeutlichen.

**Vergleich der Fähigkeiten:**

| Aspekt | Einfaches LLM | Agent |
|--------|---------------|-------|
| **Aktuelle Daten** | ❌ Nur Trainingsdaten | ✅ Über Tools |
| **Berechnungen** | ⚠️ Oft ungenau | ✅ Präzise Tools |
| **Externe APIs** | ❌ Unmöglich | ✅ Beliebig erweiterbar |
| **Transparenz** | 🔒 Verborgen | 👁️ Sichtbar |
| **Erweiterbarkeit** | ❌ Statisch | ✅ Modular |



**🎯 FAZIT:**
- **Agent = LLM + Tools + Reasoning**
- ➡️ Aus reaktiv wird proaktiv
- ➡️ Aus statisch wird dynamisch
- ➡️ Aus isoliert wird vernetzt



# 3 | Anatomie eines Agenten
---


Um Agenten effektiv einsetzen zu können, muss man  ihre innere Struktur verstehen. Ein LangGraph-Agent besteht aus Komponenten, die zusammenarbeiten, um komplexe Aufgaben zu lösen. Diese moderne Architektur ermöglicht es, die Stärken von LLMs mit praktischen Werkzeugen zu kombinieren.

**Die Kern-Komponenten (LangGraph)**

LangGraph verwendet eine **graph-basierte** Architektur statt der Legacy AgentExecutor-Struktur. Jede Komponente hat eine spezifische Rolle im Gesamtsystem.

+ **LLM (Das Gehirn)**: Das Large Language Model fungiert als zentrale Intelligenz des Agenten. Es versteht die Benutzeranfrage, interpretiert Tool-Ergebnisse und entscheidet über nächste Schritte. Ohne das LLM wäre der Agent nur eine Sammlung unverbundener Werkzeuge.

+ **Tools (Die Hände)**: Tools sind spezialisierte Funktionen, die dem Agenten erlauben, mit der Außenwelt zu interagieren. Sie können so einfach sein wie ein Rechner oder so komplex wie eine Datenbankverbindung. Jedes Tool erweitert die Fähigkeiten des Agenten erheblich.

+ **State/Messages (Das Gedächtnis)**: LangGraph verwendet einen **Message-basierten State** statt eines Scratchpads. Alle Zwischenergebnisse, Tool-Aufrufe und deren Ergebnisse werden als Messages im State gespeichert. Dies ermöglicht transparente und nachvollziehbare Reasoning-Prozesse.

+ **Graph (Der Orchestrator)**: Der Graph ist das Herzstück von LangGraph. Er definiert die Abfolge von Nodes (LLM, Tools) und Edges (Übergänge). Dies ermöglicht komplexe Workflows mit Bedingungen, Schleifen und paralleler Ausführung.


<br>

**Der Agent-*Denkprozess* (LangGraph)**

Der *Denkprozess* eines LangGraph-Agenten ist graph-basiert und **zustandsorientiert**. Statt linearem Ablauf durchläuft der Agent einen iterativen Graph-Zyklus.

+ **Schritt 1 - Verstehen**: Der Agent analysiert die Benutzeranfrage (als HumanMessage) und identifiziert, welche Informationen oder Aktionen benötigt werden. Dies geschieht durch das LLM-Node im Graph.

+ **Schritt 2 - Planen**: Basierend auf dem Verständnis der Anfrage plant der Agent, welche Tools verwendet werden sollten. In LangGraph wird dies durch **Tool-Calling** realisiert - das LLM generiert ToolMessages mit den benötigten Tool-Aufrufen.

+ **Schritt 3 - Ausführen**: Der Graph führt die Tool-Nodes aus und erhält Ergebnisse. Diese werden als ToolMessage-Objekte im State gespeichert und stehen für weitere Entscheidungen zur Verfügung.

+ **Schritt 4 - Bewerten**: Nach jedem Tool-Aufruf entscheidet der Graph (via Conditional Edges), ob genügend Informationen vorliegen oder weitere Schritte notwendig sind. Diese Bewertung bestimmt den weiteren Verlauf.

+ **Schritt 5 - Iterieren oder Antworten**: Je nach Bewertung kehrt der Graph zum LLM-Node zurück oder beendet mit einer finalen AIMessage. Diese Flexibilität ermöglicht die Lösung komplexer, unvorhersehbarer Probleme.

**Vorteile von LangGraph gegenüber Legacy AgentExecutor:**
- ✅ **Bessere Kontrolle** über den Ablauf durch explizite Graph-Struktur
- ✅ **State-Management** durch Messages statt verstecktem Scratchpad
- ✅ **Erweiterbarkeit** durch custom Nodes und Edges
- ✅ **Streaming** und **Parallelisierung** nativ unterstützt


**Zustandsgraph**


Bei `create_react_agent` ist der Graph **implizit vordefiniert** - Man kann ihn nicht direkt im Code *sehen*. LangGraph erstellt intern einen **ReAct-Graph** mit folgender Struktur:

**🔄 Der ReAct-Graph (intern)**



<img src="https://raw.githubusercontent.com/ralf-42/Image/main/agent_01.png" class="logo" width="300"/>

**Die drei Kern-Komponenten:**

1. **`agent` Node** (LLM)
   - Analysiert Anfrage und bisherigen State
   - Entscheidet: Welches Tool verwenden oder finale Antwort geben?
   - Output: AIMessage (mit oder ohne tool_calls)

2. **`tools` Node** (Tool-Executor)
   - Führt die vom LLM angeforderten Tools aus
   - Sammelt Ergebnisse parallel
   - Output: ToolMessage(s)

3. **Conditional Edge** (Router)
   - Prüft: Hat AIMessage tool_calls?
   - JA → Route zu `tools` Node
   - NEIN → Route zu END



**📊 State-Struktur**

Der State ist ein Dictionary mit einem `messages` Key:

```python
State = {
    "messages": [
        HumanMessage("Frage"),
        AIMessage("...", tool_calls=[...]),  # LLM möchte Tools nutzen
        ToolMessage("Ergebnis 1"),
        ToolMessage("Ergebnis 2"),
        AIMessage("Finale Antwort")  # Keine tool_calls → END
    ]
}
```



**🛠️ Expliziter Graph (Alternative)**

Wenn man **volle Kontrolle** möchten, kann man den Graph selbst definieren:

```python
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode

# 1. Graph definieren
graph_builder = StateGraph(MessagesState)

# 2. Nodes hinzufügen
graph_builder.add_node("agent", lambda state: {"messages": [llm.invoke(state["messages"])]})
graph_builder.add_node("tools", ToolNode(tools))

# 3. Edges definieren
graph_builder.add_edge(START, "agent")

# 4. Conditional Edge: Routing-Logik
def should_continue(state):
    last_message = state["messages"][-1]
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"  # Weiter zu Tools
    return END  # Fertig

graph_builder.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})
graph_builder.add_edge("tools", "agent")  # Nach Tools zurück zu Agent

# 5. Graph kompilieren
custom_graph = graph_builder.compile()

# Verwenden wie create_react_agent
response = custom_graph.invoke({"messages": [("human", "frage")]})
```



**📈 Vergleich**

| Aspekt | `create_react_agent` | Eigener Graph |
|--------|----------------------|---------------|
| **Einfachheit** | ✅ Eine Zeile Code | ⚠️ ~15 Zeilen |
| **Flexibilität** | ⚠️ Vordefiniert | ✅ Volle Kontrolle |
| **Graph sichtbar** | ❌ Versteckt | ✅ Explizit |
| **Custom Nodes** | ❌ Nicht möglich | ✅ Beliebig erweiterbar |
| **Best Practice** | ✅ Für Standard-Agents | ✅ Für komplexe Workflows |



**Empfehlung:**
- **Start:** `create_react_agent` für schnellen Einstieg
- **Fortgeschritten:** Eigener Graph für custom Workflows (z.B. Multi-Agent, Parallelisierung, Bedingungen)


# 4 | Hands-On: Agent mit Tools
---

In diesem Abschnitt bauen wir einen praktischen Agenten mit mehreren Tools und modernen Best Practices.

In [ ]:
# LangGraph Imports
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.utilities.serpapi import SerpAPIWrapper
import os

In [ ]:
# Tools definieren mit Pydantic Models

from pydantic import BaseModel, Field
from langchain_core.tools import StructuredTool

# ========== Pydantic Schemas ==========

class FileReadInput(BaseModel):
    """Schema für Datei-Lesen"""
    filename: str = Field(description="Pfad zur zu lesenden Datei")

class FileWriteInput(BaseModel):
    """Schema für Datei-Schreiben"""
    filename: str = Field(description="Zieldatei (z.B. 'output.txt')")
    content: str = Field(description="Zu schreibender Textinhalt")

class SearchInput(BaseModel):
    """Schema für Internet-Suche"""
    query: str = Field(description="Suchbegriff oder Frage")

class WikiSearchInput(BaseModel):
    """Schema für Wikipedia-Suche"""
    term: str = Field(description="Wikipedia-Suchbegriff")

# ========== Tool-Funktionen ==========

def read_file_func(filename):
    """Liest eine Datei und gibt den Inhalt zurück"""
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        return f"Fehler beim Lesen von {filename}: {e}"

def write_file_func(filename, content):
    """Schreibt Inhalt in eine Datei"""
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(content)
        return f"✅ Datei {filename} erfolgreich geschrieben ({len(content)} Zeichen)"
    except Exception as e:
        return f"Fehler beim Schreiben: {e}"

def search_web_func(query):
    """Internet-Suche mit robuster Fehlerbehandlung"""
    try:
        serpapi = SerpAPIWrapper()
        result = serpapi.run(query)
        return result
    except Exception as e:
        error_msg = str(e)
        if "quota" in error_msg.lower() or "limit" in error_msg.lower():
            return f"⚠️ SerpAPI Rate Limit erreicht. Fehler: {error_msg}"
        elif "api key" in error_msg.lower():
            return f"⚠️ SerpAPI Key Problem. Fehler: {error_msg}"
        else:
            return f"⚠️ Suchfehler: {error_msg}"

def wiki_search_func(term):
    """Wikipedia-Suche"""
    wiki = WikipediaAPIWrapper()
    try:
        return wiki.run(term)
    except Exception as e:
        return f"Wikipedia-Fehler: {e}"

# ========== Strukturierte Tools erstellen ==========

custom_tools = [
    StructuredTool.from_function(
        func=read_file_func,
        name="read_file",
        description="Liest den Inhalt einer Textdatei. Nutze dies, um bestehende Dateien zu lesen.",
        args_schema=FileReadInput
    ),
    StructuredTool.from_function(
        func=write_file_func,
        name="write_file",
        description="Schreibt Text in eine Datei. Nutze dies, um neue Dateien zu erstellen oder zu überschreiben.",
        args_schema=FileWriteInput
    ),
    StructuredTool.from_function(
        func=search_web_func,
        name="search",
        description="Sucht aktuelle Informationen im Internet (Aktienkurse, News, Software-Versionen, etc.). Nutze dies für zeitkritische Informationen.",
        args_schema=SearchInput
    ),
    StructuredTool.from_function(
        func=wiki_search_func,
        name="wiki",
        description="Sucht Informationen in Wikipedia. Gut für Personen, Konzepte, Geschichte, allgemeines Wissen.",
        args_schema=WikiSearchInput
    )
]

mprint("### ✅ Tools mit Pydantic Models definiert:")
for tool in custom_tools:
    print(f"   • {tool.name:15s}: {tool.description[:80]}...")

In [ ]:
# Modell definieren
# Hinweis: Bei LangGraph wird der Prompt direkt als String übergeben,
# nicht als ChatPromptTemplate wie beim Legacy AgentExecutor

model_name = "gpt-4o-mini"
temperature = 0.0

llm = ChatOpenAI(model=model_name, temperature=temperature)

In [ ]:
# LangGraph Agent erstellen
system_prompt = "Du bist ein hilfreicher Assistent mit Zugriff auf Tools."

custom_agent = create_react_agent(
    llm,
    custom_tools,
    prompt=system_prompt
)

print("✅ Agent erstellt")

In [ ]:
# 📊 Graph-Visualisierung
from IPython.display import Image, display

mprint("### 📊 Agent-Graph Visualisierung")
mprint("---")
mprint("Der interne Zustandsgraph des Agenten:")

try:
    # Graph als Mermaid-Diagramm anzeigen
    graph_image = custom_agent.get_graph().draw_mermaid_png()
    display(Image(graph_image))
except Exception as e:
    print(f"⚠️ Graph-Visualisierung nicht verfügbar: {e}")
    print("Tipp: Installiere 'pygraphviz' oder nutze Mermaid online")

**🔍 Was zeigt die Visualisierung?**

Der Graph zeigt den internen Ablauf des Agenten:

- **`__start__`** → Einstiegspunkt
- **`agent`** → LLM-Node (entscheidet über Tools)
- **`tools`** → Führt Tool-Aufrufe aus
- **`__end__`** → Beendet die Ausführung

**Ablauf:**
1. Start → Agent (LLM analysiert Anfrage)
2. Agent → Tools? (Conditional Edge prüft tool_calls)
   - JA: Zu Tools Node → zurück zu Agent
   - NEIN: Zu End (finale Antwort)

Dieser Graph ist **implizit** in `create_react_agent` vordefiniert!

In [ ]:
# Test-Anfrage
from datetime import date
today = date.today().strftime("%d.%m.%Y")

input_text = f"""
Erstelle eine Datei 'notiz.txt' mit dem Inhalt 'Agenten können autonom agieren. 🤖'.
Wie ist der aktuelle XETRA Kurs der Aktie von Rheinmetall vom {today}?
Lese die Datei 'notiz.txt' und verändere die Zeitform von Gegenwart in Zukunft (speichere als 'notiz_zukunft.txt').
Was steht zu Taylor Swift auf Wikipedia (nur die erste Zusammenfassung)?
"""

print("📝 Test-Anfrage erstellt")

In [ ]:
# LangGraph verwendet messages-basierte Eingabe
response = custom_agent.invoke({"messages": [("human", input_text)]})

In [ ]:
mprint("## 🛠️ Hands-On Agent (LangGraph)")
mprint("---")
mprint("**Input:**")
mprint(input_text)
mprint("**Output**:")
# LangGraph speichert alle Messages im State - letzte Message ist die Antwort
output_message = response["messages"][-1].content
mprint(output_message)

In [ ]:
# Alle messages
for i, item in enumerate(response["messages"],1):
    mprint(f"**>> {i}. <<**")
    print(f"{item.content:50s} ...")

# 5 | Wann braucht man Agenten?
---

Die Entscheidung zwischen einem einfachen LLM und einem Agenten hängt von den spezifischen Anforderungen Ihrer Anwendung ab. Eine klare Entscheidungsmatrix hilft dabei, die richtige Technologie für den jeweiligen Anwendungsfall zu wählen und Ressourcen effizient einzusetzen.



Die Wahl der richtigen Technologie beginnt mit der Analyse der Aufgabenanforderungen. Während LLMs für viele Textverarbeitungsaufgaben ausreichen, sind Agenten unverzichtbar, wenn externe Interaktionen oder aktuelle Daten benötigt werden.

**Verwenden Sie einen Agenten wenn:**

+ Sie **aktuelle** oder **dynamische** Daten benötigen, die sich häufig ändern (Aktienkurse, Wetter, Nachrichten). Agenten können über APIs auf Live-Daten zugreifen und diese in ihre Antworten integrieren.

+ Präzise Berechnungen erforderlich sind, bei denen **Genauigkeit** kritisch ist. LLMs approximieren mathematische Operationen, während Agenten echte Rechner-Tools verwenden.

+ **Externe** Systeme angesprochen werden müssen, wie Datenbanken, APIs oder andere Services. Agenten können diese Integrationen nahtlos abwickeln.

+ Komplexe, **mehrstufige Prozesse** durchgeführt werden sollen, bei denen jeder Schritt vom vorherigen abhängt. Der Agent-Reasoning-Loop ist für solche Szenarien optimiert.

**Ein einfaches LLM reicht wenn:**

+ Reine Textverarbeitung ohne externe Daten im Fokus steht. Für Zusammenfassungen, Übersetzungen oder Textanalysen sind LLMs optimal.

+ Kreative Aufgaben gelöst werden sollen, wie das Schreiben von Geschichten, Gedichten oder Marketing-Texten. Hier sind die kreativen Fähigkeiten des LLMs gefragt.

+ Erklärungen oder Bildungsinhalt basierend auf allgemeinem Wissen benötigt werden. LLMs haben Zugang zu einem enormen Wissensfundus.

+ Statische Code-Generierung ohne externe Abhängigkeiten erforderlich ist. Für einfache Programmieraufgaben sind LLMs sehr effektiv.

<br>

**Die Faustregel lautet**:    
Wenn Sie Tools, aktuelle Daten oder externe Interaktionen benötigen, wählen Sie einen Agenten. Für reine Textverarbeitung reicht ein LLM aus.

# A | Aufgabe
---

Die Aufgabestellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.

<p><font color='black' size="5">
Kalkulation
</font></p>

Gegeben ist eine Datei, die eine Reihe von Gleichungen enthält.
Der Dateiname ist GenAI/02 data/gleichungen.txt

**Gleichung:**    
41748459 - 87226336    
92995162 * 46769739    
61530438 * 56074589    
95329602 + 45418854    
412907 + 3731910    
...

Verwenden Sie einen LangChain-Agenten mit einem Tool, um jede dieser Gleichungen zu berechnen, und erstellen Sie eine Datei ähnlich dieser:

**Ergebnisse:**  
41748459 - 87226336 = 45477877   
92995162 * 46769739 = 4349359455002718   
61530438 * 56074589 = 3450294021839982   
95329602 + 45418854 = 140748456   
412907 + 3731910 = 4144817   
... ...





# B | Exkurs: Debugging & Transparenz
---



💡 "Wie sehe ich den Denkprozess des Agenten?"

In LangGraph gibt es **keinen `verbose` Parameter** wie beim alten `AgentExecutor`. Stattdessen gibt es **modernere und bessere** Methoden:

**🔄 Option 1: Streaming (Empfohlen)**

**Live-Feedback während der Ausführung:**

```python
for event in agent.stream(
    {"messages": [("human", "frage")]},
    stream_mode="values"  # Zeigt alle Zwischenschritte
):
    last_message = event["messages"][-1]
    
    # Tool-Aufrufe
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        for tool in last_message.tool_calls:
            print(f"🔧 Tool: {tool['name']}")
    
    # Tool-Ergebnisse
    if last_message.type == "tool":
        print(f"📊 Ergebnis: {last_message.content}")
```

**Andere stream_mode Optionen:**
- `"values"` - Zeigt kompletten State nach jedem Schritt
- `"updates"` - Zeigt nur Änderungen
- `"messages"` - Zeigt nur neue Messages


**🐛 Option 2: Debug-Mode**

**Für Entwicklung und Testing:**

```python
agent = create_react_agent(
    llm,
    tools,
    prompt=system_prompt,
    debug=True  # ✅ Aktiviert interne Logs
)
```

**📊 Option 3: LangSmith (Production-Grade)**

**Professionelles Tracing und Monitoring:**

```python
import os

# LangSmith aktivieren
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "your-key"  # Von smith.langchain.com

# Jetzt werden ALLE Schritte automatisch geloggt
response = agent.invoke({"messages": [("human", "frage")]})

# ➡️ Siehe Traces auf: https://smith.langchain.com
```

**LangSmith Features:**
- ✅ Komplette Execution Traces
- ✅ Token-Usage Tracking
- ✅ Latency-Monitoring
- ✅ Error-Tracking
- ✅ A/B Testing von Prompts

**📈 Vergleich der Optionen**

| Methode | Use Case | Vorteile | Nachteile |
|---------|----------|----------|-----------|
| **Streaming** | Entwicklung, Live-Demo | Echtzeit-Feedback | Mehr Code |
| **Debug-Mode** | Schnelles Debugging | Einfach zu aktivieren | Weniger Details |
| **LangSmith** | Production, Analytics | Umfassende Insights | Erfordert Setup |

**Empfehlung:**
- **Entwicklung:** Streaming für interaktive Demos
- **Debugging:** Debug-Mode für schnelle Fehlersuche  
- **Production:** LangSmith für professionelles Monitoring
